In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import glob
import os
import warnings

from astropy.io import ascii
from astropy.io import fits

%matplotlib inline

In [3]:
igrins_log = pd.read_csv('./reduced_log.csv') # RRSIA reduced data log
# igrins_log = igrins_log.drop(labels=['SNRH','SNRK','ROTPA','AM','BVC'],axis=1) # drop a few columns

luhman_cat = pd.read_table('./apjs328021t7_sptype_ascii.txt',header=0) # luhman data table
luhman_sptypes = luhman_cat['SED_Class'] # SED class column

class_3_mask = luhman_sptypes == 'III' # conditional statement
luhman_class3 = luhman_cat[class_3_mask] # apply mask
luhman_class3 # all class 3 objects in Luhman's catalog (2008 I think?)

,2MASS,Name,Spectral_Type,alpha(2-8 mum),alpha(2-24 mum),alpha(3.6-8 mum),alpha(3.6-24 mum),SED_Class,Unnamed: 8
0,J04034930+2610520,HBC 358 A+B+C,M3.5,-2.49/-2.49,-2.68/-2.68,-2.74/-2.74,-2.83/-2.83,III,NaN
2,J04035084+2610531,HBC 359,M2,-2.58/-2.58,-2.77/-2.77,-2.74/-2.74,-2.88/-2.88,III,NaN
3,J04043936+2158186,HBC 360,M3.5,-2.54/-2.55,...,-2.75/-2.75,...,III,NaN
4,J04043984+2158215,HBC 361,M3,-2.55/-2.55,...,-2.73/-2.73,...,III,NaN
6,J04053087+2151106,HBC 362,M2,-2.58/-2.58,-2.63/-2.63,-2.77/-2.77,-2.73/-2.73,III,NaN
...,...,...,...,...,...,...,...,...,...
340,J04560201+3021037,HBC 427,K5,-2.72/-2.75,-2.69/-2.71,-2.77/-2.79,-2.71/-2.72,III,NaN
341,J04574903+3015195,...,M9.25,-2.29/-2.29,...,-2.76/-2.76,...,III,NaN
346,J05061674+2446102,CIDA 10,M4,-2.50/-2.54,...,-2.66/-2.67,...,III,NaN
348,J05064662+2104296,...,M5.25,-2.40/-2.42,...,-2.67/-2.68,...,III,NaN


In [ ]:
luhman_class3

,2MASS,Name,Spectral_Type,alpha(2-8 mum),alpha(2-24 mum),alpha(3.6-8 mum),alpha(3.6-24 mum),SED_Class,Unnamed: 8
0,J04034930+2610520,HBC 358 A+B+C,M3.5,-2.49/-2.49,-2.68/-2.68,-2.74/-2.74,-2.83/-2.83,III,NaN
2,J04035084+2610531,HBC 359,M2,-2.58/-2.58,-2.77/-2.77,-2.74/-2.74,-2.88/-2.88,III,NaN
3,J04043936+2158186,HBC 360,M3.5,-2.54/-2.55,...,-2.75/-2.75,...,III,NaN
4,J04043984+2158215,HBC 361,M3,-2.55/-2.55,...,-2.73/-2.73,...,III,NaN
6,J04053087+2151106,HBC 362,M2,-2.58/-2.58,-2.63/-2.63,-2.77/-2.77,-2.73/-2.73,III,NaN
...,...,...,...,...,...,...,...,...,...
340,J04560201+3021037,HBC 427,K5,-2.72/-2.75,-2.69/-2.71,-2.77/-2.79,-2.71/-2.72,III,NaN
341,J04574903+3015195,...,M9.25,-2.29/-2.29,...,-2.76/-2.76,...,III,NaN
346,J05061674+2446102,CIDA 10,M4,-2.50/-2.54,...,-2.66/-2.67,...,III,NaN
348,J05064662+2104296,...,M5.25,-2.40/-2.42,...,-2.67/-2.68,...,III,NaN


In [ ]:
# unique targets of observation from RRISA
unique_rrisa = pd.Series(igrins_log['OBJNAME_super'].unique())
unique_rrisa

0            LHS 3325
1                vega
2           NLTT45791
3           NLTT46858
4              GJ797B
            ...      
5264           AQ PUP
5265        HIP 46633
5266           AQ-CAR
5267           YZ-CAR
5268    TIC 168426455
Length: 5269, dtype: object

In [ ]:
match_mask = luhman_class3['Name'].isin(unique_rrisa) # check the names from RRISA against Luhman class 3 objects
luhman_class3 = luhman_class3[match_mask].sort_values(['Spectral_Type']) # just some sorting

class3_objects = luhman_class3['Name'] # just the names of class 3 objects that are also in RRISA (allegedly)

In [ ]:
luhman_class3.iloc

NameError: name 'luhman_class3' is not defined

In [ ]:
import requests
import ast

igrins_class3 = igrins_log[igrins_log['OBJNAME_super'].isin(luhman_class3['Name'])]
igrins_class3['URLS_K']

430     {'20141125/SDCK_20141125_0127.sn.fits': 'https...
461     {'20141127/SDCK_20141127_0104.sn.fits': 'https...
463     {'20141127/SDCK_20141127_0112.sn.fits': 'https...
518     {'20141129/SDCK_20141129_0129.sn.fits': 'https...
530     {'20141130/SDCK_20141130_0145.sn.fits': 'https...
                              ...                        
8921    {'20190309/SDCK_20190309_0035.sn.fits': 'https...
8924    {'20190309/SDCK_20190309_0041.sn.fits': 'https...
8934    {'20190314/SDCK_20190314_0039.sn.fits': 'https...
8943    {'20190316/SDCK_20190316_0033.sn.fits': 'https...
8947    {'20190316/SDCK_20190316_0041.sn.fits': 'https...
Name: URLS_K, Length: 280, dtype: object

In [ ]:
print(igrins_class3['URLS_K'].values[0])

{'20141125/SDCK_20141125_0127.sn.fits': 'https://utexas.box.com/shared/static/ai78krotop15gcr049zuo7ukq9pk4g4i.fits', '20141125/SDCK_20141125_0127.spec.fits': 'https://utexas.box.com/shared/static/ev0i9qoj8tetg18aknafyihth0ei80sb.fits', '20141125/SDCK_20141125_0127.spec_a0v.fits': 'https://utexas.box.com/shared/static/gzpkmo33fqe8xun9zkj68krhcsgq3zxf.fits', '20141125/SDCK_20141125_0127.variance.fits': 'https://utexas.box.com/shared/static/5dldlxeisz56je3cx8ayf67e2qk9sssy.fits'}


In [ ]:
# #using a session here means that requests can fetch the information faster
# #since the connection is reused
# session = requests.Session()
# #the response holds lots of information gathered from the request (session.get())
# response = session.get(test_dict['20170915/SDCH_20170915_0049.spec_a0v.fits'])
# #we can check the status code to make sure our link was found successfully
# if response.status_code == 200:
#   #open a file with the corresponding filename to dump the file content into
#   with open("SDCH_20170915_0049.spec_a0v.fits", 'wb') as f:
#     #write the file
#     f.write(response.content)
#   #close the file--especially important when writing many files at once or
#   #trying to prevent against corrupted files
#   f.close()
#   #print that we downloaded the file
#   print(f"Downloaded SDCH_20170915_0049.spec_a0v.fits")